In [7]:
import json
import difflib
import copy
from pprint import pprint
#Reading the file
f = open('python1007.json')
with open('python1007.json',encoding='utf-8') as f:
    content = f.readlines()
#Getting rid of empty strings
content =  [x.strip() for x in content]
content = list(filter(None, content)) 
TweetsJsonArray = []
#To JSON TYPE
for tweet in content:
    TweetsJsonArray.append(json.loads(tweet))  
#Preprocessing - deleting the similar tweets- deciede that it is not needed
import difflib
def similar(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

In [8]:
from html.parser import HTMLParser

In [9]:
#Writing the content to the exel for Manual Analysis
import re
import csv
import json
import xlwt
import re

num=1;
book = xlwt.Workbook()
sheet1 = book.add_sheet("Sheet1",cell_overwrite_ok=True)

row = sheet1.row(0)

row.write(0, 'Created at')
row.write(1, 'Text')  
row.write(2, 'Names') 
row.write(3, 'Description') 
row.write(4, 'Locations') 
row.write(5, 'Id') 
n=0
for tweet in TweetsJsonArray: 
    n=n+1
    if tweet['lang']=="en":
        row = sheet1.row(num)
        row.write(0, tweet['created_at'])  
        text=''
        if 'extended_tweet' in tweet:
            text=tweet['extended_tweet']['full_text']
        elif 'retweeted_status' in tweet:
            if 'extended_tweet' in tweet['retweeted_status']:
                text=tweet['retweeted_status']['extended_tweet']['full_text']
            else:
                text=tweet['retweeted_status']['text']
        else:
            text=tweet['text']
        text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
        names = tweet['user']['name']
        description = tweet['user']['description']
        #text = re.sub(r'\W+', '', text)
        
        PERMITTED_CHARS = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_- "
        text = "".join(c for c in text if c in PERMITTED_CHARS)
        text=text.lower()
        text=text.strip()
        names="".join(c for c in names if c in PERMITTED_CHARS)
        description="".join(c for c in description if c in PERMITTED_CHARS) if description!= None else ''
        #text=text+";\n"+description       
        num=num+1              
        
        row.write(1, text)
        row.write(2, names)
        row.write(3, description)
        row.write(4, tweet['user']['location'] if tweet['user']['location'] != None else '')
        row.write(5, num)
        num=num+1
book.save("TweetCorpusExel3.xls")
    #csvFile.writerow([tweet["created_at"],tweet["text"],tweet["user"]["name"],tweet["user"]["description"],tweet["user"]["location"]])

In [10]:
import pandas as pd

xl = pd.ExcelFile("TweetCorpusExel3.xls")
df = xl.parse("Sheet1")
print (df['Text'][2])
df = df.sort_values(by="Text")
df['TextDescr'] = df.Text.map(str) + " :: " + df.Description

writer = pd.ExcelWriter('TweetCorpusExel4.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()
writer.close()

these victims are criminals illegally in our country


In [11]:
df = pd.read_excel('TweetCorpusExel4.xlsx', sheetname='Sheet1')
writer = pd.ExcelWriter('Final.xlsx')
currentrow=0
tweet=df['Text'][currentrow]
l = len(df)
for i in range(l):   
    if df['Text'][i]==tweet or similar(df['Text'][i],tweet)>0.8:
        df['Text'][i]=''        
        df['Id'][currentrow]=str(df['Id'][currentrow])+","+str(df['Id'][i])
        if df['Names'][i]!='':
            df['Names'][currentrow]=str(df['Names'][currentrow]) +","+str(df['Names'][i])
        df['Text'][currentrow]=str(df['Text'][currentrow])+" \n ; "+str(df['Description'][currentrow])
        df['Locations'][currentrow]=str(df['Locations'][currentrow])+","+str(df['Locations'][i])
    else:
        currentrow=i
        tweet=df['Text'][currentrow]
df = df[df.Text != '']

del df['Text']
#len(filter(lambda df: df.TextDescr.str.contains(r'@'),rows))
del df['Description']
#df['Text'][currentrow]=str(df['Text'][currentrow])+" \n ; "+str(df['Description'][currentrow])
df.to_excel(writer,'Sheet1',index=False)

writer.save()
writer.close()

/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
json.dumps(TweetsJsonArray[1]['user'], sort_keys=True, indent=4)

'{\n    "contributors_enabled": false,\n    "created_at": "Mon Dec 21 17:32:31 +0000 2015",\n    "default_profile": true,\n    "default_profile_image": false,\n    "description": null,\n    "favourites_count": 96046,\n    "follow_request_sent": null,\n    "followers_count": 733,\n    "following": null,\n    "friends_count": 196,\n    "geo_enabled": true,\n    "id": 4559222536,\n    "id_str": "4559222536",\n    "is_translator": false,\n    "lang": "en-gb",\n    "listed_count": 28,\n    "location": null,\n    "name": "sylvia moulds",\n    "notifications": null,\n    "profile_background_color": "F5F8FA",\n    "profile_background_image_url": "",\n    "profile_background_image_url_https": "",\n    "profile_background_tile": false,\n    "profile_image_url": "http://pbs.twimg.com/profile_images/792728496300711936/u5VdEbmc_normal.jpg",\n    "profile_image_url_https": "https://pbs.twimg.com/profile_images/792728496300711936/u5VdEbmc_normal.jpg",\n    "profile_link_color": "1DA1F2",\n    "profil

In [95]:
from csv import DictReader

with open('bad-words.csv', 'rb') as f:
    bad = [row["rowname"] for row in DictReader(f)]#edit this row name
    value = [row["rowname"] for row in DictReader(f)]  # edit this row name
filename = sys.argv[1]
row_number, column_number = [int(arg, 10)-1 for arg in sys.argv[2:]]
with open(filename, 'rb') as g:
     rows = list(csv.reader(f))
     print (rows[1][4])#give for loop to do this automatically for all strings
var1  = rows[row_number][column_number].split(',')
list(set(f).intersection(set(var1))

SyntaxError: unexpected EOF while parsing (<ipython-input-95-d00dd8ff3fe4>, line 12)